In [1]:
import pandas as pd
import numpy as np
import random
import os
import torch
import sys
sys.path.append('../')
from modules import utils, constants

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

In [3]:
testing_df = pd.read_csv('../data/18_jan/test_set_basic.csv')
testing_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,0,0,0,0,1,0,1,0,0,...,1,0,0,1,0,1,0,0,0,1
1,1,0,0,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,1,0,1
2,1,0,1,1,0,1,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
3,1,1,0,0,0,1,0,0,1,0,...,0,1,1,1,0,0,0,0,0,1
4,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
len(testing_df[testing_df.ana == 0])

3988

In [5]:
testing_df.tail()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
13995,1,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
13996,1,1,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
13997,1,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
13998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13999,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [6]:
X_test = testing_df.iloc[:, 0:-1]
y_test = testing_df.iloc[:, -1]

X_test, y_test = np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((14000, 23), (14000,))

In [11]:
dqn_model = utils.load_dqn3('../models/18_jan/dqn_basic_12000000')
test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
test_df.head()

Using stable baselines 3


KeyboardInterrupt: 

In [15]:
test_df.episode_length.min(), test_df.episode_length.max()

(5.0, 10.0)

In [16]:
test_df[test_df.episode_length == 4]

,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,score,is_success


In [17]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

86.0

In [18]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(6.702285714285714, 0.72)

In [19]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(86.0, 85.6961454087426, 85.60721960941125)

In [20]:
test_df.y_pred.value_counts()

1.0    8871
0.0    5119
2.0      10
Name: y_pred, dtype: int64

In [29]:
test_df.y_actual.value_counts()

1.0    7247
0.0    6753
Name: y_actual, dtype: int64

#### Saving files

In [ ]:
#test_df.to_csv(f'../test_dfs/dqn_test_df_balanced_14000000.csv', index=False)
#success_df.to_csv(f'../test_dfs/dqn_success_df_balanced_14000000.csv', index=False)